### To DO:
- Hyperparameters for optimizers
- Possibly test all parameters at the same time
- Apply best to CIFAR

In [1]:
#! pip install keras
#! pip install tensorflow
#! pip install torch
! pip install tensorflow[and-cuda]

INFO: pip is looking at multiple versions of tensorflow[and-cuda] to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for tensorflow[and-cuda] from https://files.pythonhosted.org/packages/cd/4c/9ccf0ac5fd00372146890b801c9b9912269ffe0d208115b75c7dc11e7edc/tensorflow-2.13.1-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for tensorflow-macos==2.13.1 from https://files.pythonhosted.org/packages/c0/d1/d309dea6e67e1b8037f607872486eb67a1ff64fb91a96149086dbdc46ca4/tensorflow_macos-2.13.1-cp311-cp311-macosx_12_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 10.0 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for tensorflow-estimator<2.14,>=2.13.0 from https://files.pythonhosted.org/packages/72/5c/c318268d96791c6222ad7df1651bbd1b2409139afeb6f468c0f327177016/tensorflow_estimator-2.13.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information 

In [2]:
import keras as keras
from __future__ import print_function
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
import numpy as np
import pandas as pd


# initializers
from keras.initializers import Zeros  
from keras.initializers import RandomNormal, RandomUniform  
from keras.initializers import glorot_normal, glorot_uniform 
from keras.initializers import he_normal, he_uniform  
from keras.initializers import lecun_normal, lecun_uniform  


# Task 1.1 

In [3]:
# mnist_mlp.py
batch_size = 128
num_classes = 10
epochs = 20

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

60000 train samples
10000 test samples
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20


2023-10-25 11:16:47.306747: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


469/469 [==============================] - 2s 4ms/step - loss: 0.2429 - accuracy: 0.9250 - val_loss: 0.1168 - val_accuracy: 0.9647
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1045 - accuracy: 0.9685 - val_loss: 0.0815 - val_accuracy: 0.9750
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0747 - accuracy: 0.9775 - val_loss: 0.0691 - val_accuracy: 0.9798
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0589 - accuracy: 0.9820 - val_loss: 0.0704 - val_accuracy: 0.9820
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0513 - accuracy: 0.9850 - val_loss: 0.0726 - val_accuracy: 0.9809
Epoch 6/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0430 - accuracy: 0.9875 - val_loss: 0.0761 - val_accuracy: 0.9806
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0398 - accuracy: 0.9885 - val_loss: 0.0818 - val_accuracy: 0.9823
Epoch 8/20
469/46

In [4]:
# mnist_cnn.py
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/12
469/469 [==============================] - 25s 52ms/step - loss: 2.2751 - accuracy: 0.1553 - val_loss: 2.2429 - val_accuracy: 0.3103
Epoch 2/12
469/469 [==============================] - 26s 55ms/step - loss: 2.2230 - accuracy: 0.2540 - val_loss: 2.1777 - val_accuracy: 0.4576
Epoch 3/12
469/469 [==============================] - 26s 55ms/step - loss: 2.1542 - accuracy: 0.3497 - val_loss: 2.0890 - val_accuracy: 0.5476
Epoch 4/12
469/469 [==============================] - 29s 61ms/step - loss: 2.0601 - accuracy: 0.4325 - val_loss: 1.9708 - val_accuracy: 0.6110
Epoch 5/12
469/469 [==============================] - 29s 63ms/step - loss: 1.9393 - accuracy: 0.4958 - val_loss: 1.8177 - val_accuracy: 0.6687
Epoch 6/12
469/469 [==============================] - 32s 69ms/step - loss: 1.7902 - accuracy: 0.5430 - val_loss: 1.6320 - val_accuracy: 0.7102
Epoch 7/12
469/469 [==============================] - 32s 68ms/

# Task  1.2

In [7]:
# mnist_mlp.py + different initilization methods
batch_size = 128
num_classes = 10
epochs = 20

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
seed_value = 42
results = pd.DataFrame(columns=['Initialization Method', 'Test Loss', 'Test Accuracy'])
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

initilization_methods = [Zeros(), 
                       RandomNormal(seed=seed_value), 
                       RandomUniform(seed=seed_value), 
                       glorot_uniform(seed=seed_value), 
                       glorot_normal(seed=seed_value), 
                       he_normal(seed=seed_value), 
                       he_uniform(seed=seed_value), 
                       lecun_normal(seed=seed_value), 
                       lecun_uniform(seed=seed_value)]

for method in initilization_methods:
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,), kernel_initializer=method))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu', kernel_initializer=method))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax', kernel_initializer=method))

    # Extract name of method
    method_name = method.__class__.__name__
    
    model.summary()

    
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))

    
    score = model.evaluate(x_test, y_test, verbose=0)
    
    new_result = pd.DataFrame({'Initialization Method': [method_name], 'Test Loss': [score[0]], 'Test Accuracy': [score[1]]})
    results = pd.concat([results, new_result], ignore_index=True)
    
    print(results)


60000 train samples
10000 test samples
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 512)               262656    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2

/var/folders/28/tw9kr3yd4h950rswblzvqc9h0000gn/T/ipykernel_46211/1700375211.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_result], ignore_index=True)


469/469 [==============================] - 2s 5ms/step - loss: 0.2516 - accuracy: 0.9227 - val_loss: 0.1124 - val_accuracy: 0.9649
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1039 - accuracy: 0.9684 - val_loss: 0.0825 - val_accuracy: 0.9750
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0743 - accuracy: 0.9771 - val_loss: 0.0771 - val_accuracy: 0.9768
Epoch 4/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0606 - accuracy: 0.9811 - val_loss: 0.0750 - val_accuracy: 0.9793
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0500 - accuracy: 0.9850 - val_loss: 0.0857 - val_accuracy: 0.9774
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0425 - accuracy: 0.9873 - val_loss: 0.0765 - val_accuracy: 0.9817
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0366 - accuracy: 0.9888 - val_loss: 0.0801 - val_accuracy: 0.9812
Epoch 8/20
469/46

In [9]:
# mnist_mlp.py + different activation functions
batch_size = 128
num_classes = 10
epochs = 20

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
seed_value = 42
results = pd.DataFrame(columns=['Activation Method', 'Test Loss', 'Test Accuracy'])

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

activation_methods = ['relu', 'sigmoid', 'tanh', 'linear', 'softmax']



for method in activation_methods:
    model = Sequential()
    model.add(Dense(512, activation=method, input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation=method))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))

    
    method_name = method
    
    model.summary()
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])
    
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    
    score = model.evaluate(x_test, y_test, verbose=0)
    new_result = pd.DataFrame({'Activation Method': [method_name], 'Test Loss': [score[0]], 'Test Accuracy': [score[1]]})
    results = pd.concat([results, new_result], ignore_index=True)
    
    print(results)


60000 train samples
10000 test samples
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 512)               401920    
                                                                 
 dropout_26 (Dropout)        (None, 512)               0         
                                                                 
 dense_39 (Dense)            (None, 512)               262656    
                                                                 
 dropout_27 (Dropout)        (None, 512)               0         
                                                                 
 dense_40 (Dense)            (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Epoch 1/

/var/folders/28/tw9kr3yd4h950rswblzvqc9h0000gn/T/ipykernel_46211/1699284994.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_result], ignore_index=True)


469/469 [==============================] - 2s 4ms/step - loss: 0.5377 - accuracy: 0.8336 - val_loss: 0.2651 - val_accuracy: 0.9186
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.2372 - accuracy: 0.9274 - val_loss: 0.1764 - val_accuracy: 0.9448
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1693 - accuracy: 0.9487 - val_loss: 0.1353 - val_accuracy: 0.9575
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1302 - accuracy: 0.9599 - val_loss: 0.1071 - val_accuracy: 0.9661
Epoch 5/20
469/469 [==============================] - 2s 4ms/step - loss: 0.1077 - accuracy: 0.9668 - val_loss: 0.0912 - val_accuracy: 0.9705
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0912 - accuracy: 0.9722 - val_loss: 0.0980 - val_accuracy: 0.9700
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0797 - accuracy: 0.9755 - val_loss: 0.0777 - val_accuracy: 0.9762
Epoch 8/20
469/46

In [12]:
# mnist_mlp.py + different optimizers
batch_size = 128
num_classes = 10
epochs = 20

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
seed_value = 42
results = pd.DataFrame(columns=['Optimizer', 'Test Loss', 'Test Accuracy'])

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

optimizers = [keras.optimizers.Adam, keras.optimizers.SGD, keras.optimizers.RMSprop]


for optimizer in optimizers:
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))

    # Use the optimizer's name as method_name
    method_name = optimizer.__name__
    
    model.summary()
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer(),
                  metrics=['accuracy'])
    
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    
    score = model.evaluate(x_test, y_test, verbose=0)
    new_result = pd.DataFrame({'Optimizer': [method_name], 'Test Loss': [score[0]], 'Test Accuracy': [score[1]]})
    results = pd.concat([results, new_result], ignore_index=True)
    
    print(results)


60000 train samples
10000 test samples
Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_56 (Dense)            (None, 512)               401920    
                                                                 
 dropout_38 (Dropout)        (None, 512)               0         
                                                                 
 dense_57 (Dense)            (None, 512)               262656    
                                                                 
 dropout_39 (Dropout)        (None, 512)               0         
                                                                 
 dense_58 (Dense)            (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Epoch 1/

/var/folders/28/tw9kr3yd4h950rswblzvqc9h0000gn/T/ipykernel_46211/1283481495.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_result], ignore_index=True)


469/469 [==============================] - 2s 4ms/step - loss: 1.2457 - accuracy: 0.6788 - val_loss: 0.5536 - val_accuracy: 0.8710
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.5371 - accuracy: 0.8493 - val_loss: 0.3799 - val_accuracy: 0.8997
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 0.4273 - accuracy: 0.8769 - val_loss: 0.3245 - val_accuracy: 0.9090
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.3772 - accuracy: 0.8909 - val_loss: 0.2941 - val_accuracy: 0.9163
Epoch 5/20
469/469 [==============================] - 2s 3ms/step - loss: 0.3439 - accuracy: 0.8996 - val_loss: 0.2735 - val_accuracy: 0.9221
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.3178 - accuracy: 0.9083 - val_loss: 0.2549 - val_accuracy: 0.9282
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.3000 - accuracy: 0.9137 - val_loss: 0.2399 - val_accuracy: 0.9330
Epoch 8/20
469/46

In [14]:
# mnist_mlp.py + different regularization techniques
batch_size = 128
num_classes = 10
epochs = 20

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape (10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
seed_value = 42
results = pd.DataFrame(columns=['Regularization Method', 'Test Loss', 'Test Accuracy'])

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

regularization_methods = ['None', 'L1', 'L2', 'Dropout']

for method in regularization_methods:
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))

    if method == 'L1':
        model.add(Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l1(0.01)))
    elif method == 'L2':
        model.add(Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)))
    elif method == 'Dropout':
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.2))

    model.add(Dense(num_classes, activation='softmax'))

    # Use the regularization method as method_name
    method_name = method

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))

    score = model.evaluate(x_test, y_test, verbose=0)
    
    new_result = pd.DataFrame({'Regularization Method': [method_name], 'Test Loss': [score[0]], 'Test Accuracy': [score[1]]})
    results = pd.concat([results, new_result], ignore_index=True)

    print(results)


60000 train samples
10000 test samples
Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_67 (Dense)            (None, 512)               401920    
                                                                 
 dense_68 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
469/469 [==============================] - 1s 2ms/step - loss: 0.2565 - accuracy: 0.9252 - val_loss: 0.1259 - val_accuracy: 0.9623
Epoch 2/20
469/469 [==============================] - 1s 2ms/step - loss: 0.1039 - accuracy: 0.9696 - val_loss: 0.0862 - val_accuracy: 0.9737
Epoch 3/20
469/469 [==============================] - 1s 2ms/step - loss: 0.0691 - accuracy: 0.9789 - val_loss: 0.0785 -

/var/folders/28/tw9kr3yd4h950rswblzvqc9h0000gn/T/ipykernel_46211/400663256.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_result], ignore_index=True)


469/469 [==============================] - 2s 4ms/step - loss: 6.5434 - accuracy: 0.8136 - val_loss: 1.9698 - val_accuracy: 0.8859
Epoch 2/20
469/469 [==============================] - 2s 3ms/step - loss: 1.8520 - accuracy: 0.9057 - val_loss: 1.7738 - val_accuracy: 0.9181
Epoch 3/20
469/469 [==============================] - 2s 3ms/step - loss: 1.7281 - accuracy: 0.9261 - val_loss: 1.6590 - val_accuracy: 0.9393
Epoch 4/20
469/469 [==============================] - 2s 3ms/step - loss: 1.6650 - accuracy: 0.9365 - val_loss: 1.6664 - val_accuracy: 0.9340
Epoch 5/20
469/469 [==============================] - 2s 3ms/step - loss: 1.6227 - accuracy: 0.9452 - val_loss: 1.6120 - val_accuracy: 0.9494
Epoch 6/20
469/469 [==============================] - 2s 3ms/step - loss: 1.5930 - accuracy: 0.9508 - val_loss: 1.5967 - val_accuracy: 0.9472
Epoch 7/20
469/469 [==============================] - 2s 3ms/step - loss: 1.5721 - accuracy: 0.9538 - val_loss: 1.5608 - val_accuracy: 0.9550
Epoch 8/20
469/46

In [17]:
# mnist_mlp.py + different regularization techniques
num_classes = 10
epochs = 20

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Define hyperparameter values to test
learning_rates = [0.00001, 0.1]
momentum_values = [0, 0.9]
epsilon_values = [1e-8, 1e-4]
nesterov_values = [False, True]
batch_sizes = [64, 256]
optimizers = [keras.optimizers.Adam, keras.optimizers.SGD, keras.optimizers.RMSprop]

# Create a DataFrame to store results
results = pd.DataFrame(columns=['Optimizer', 'Learning Rate', 'Momentum', 'Epsilon', 'Nesterov', 'Batch Size', 'Test Loss', 'Test Accuracy'])

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Test different hyperparameters
for optimizer in optimizers:
    for lr in learning_rates:
        for momentum in momentum_values:
            for epsilon in epsilon_values:
                for nesterov in nesterov_values:
                    for batch_size in batch_sizes:
                        model = Sequential()
                        model.add(Dense(512, activation='relu', input_shape=(784,)))
                        model.add(Dropout(0.2))
                        model.add(Dense(512, activation='relu'))
                        model.add(Dropout(0.2))
                        model.add(Dense(10, activation='softmax'))

                        optimizer_name = optimizer.__name__
                        optimizer_instance = optimizer(learning_rate=lr)

                        if optimizer_name == 'SGD':
                            optimizer_instance.momentum = momentum
                            optimizer_instance.nesterov = nesterov
                        elif optimizer_name == 'RMSprop':
                            optimizer_instance.epsilon = epsilon

                        model.compile(loss='categorical_crossentropy',
                                      optimizer=optimizer_instance,
                                      metrics=['accuracy'])

                        history = model.fit(x_train, y_train, batch_size=batch_size, epochs=5, verbose=0, validation_data=(x_test, y_test))
                        score = model.evaluate(x_test, y_test, verbose=0)

                        new_result = pd.DataFrame({'Optimizer': [optimizer_name], 'Learning Rate': [lr], 'Momentum': [momentum], 'Epsilon': [epsilon], 'Nesterov': [nesterov], 'Batch Size': [batch_size], 'Test Loss': [score[0]], 'Test Accuracy': [score[1]]})
                        results = pd.concat([results, new_result], ignore_index=True)



# Display the results
pd.set_option('display.max_rows', None)  # To display all rows
pd.set_option('display.max_columns', None)  # To display all columns
best_results = results.sort_values(by='Test Accuracy', ascending=False).head(5)
print("Top 5 Best Performing Configurations:")
print(best_results)
print(results)


/var/folders/28/tw9kr3yd4h950rswblzvqc9h0000gn/T/ipykernel_46211/1166043142.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_result], ignore_index=True)


Top 5 Best Performing Configurations:
   Optimizer  Learning Rate Momentum       Epsilon Nesterov Batch Size  \
62       SGD            0.1      0.9  1.000000e-04     True         64   
60       SGD            0.1      0.9  1.000000e-04    False         64   
58       SGD            0.1      0.9  1.000000e-08     True         64   
50       SGD            0.1        0  1.000000e-08     True         64   
54       SGD            0.1        0  1.000000e-04     True         64   

    Test Loss  Test Accuracy  
62   0.072707         0.9780  
60   0.073003         0.9778  
58   0.070178         0.9777  
50   0.073058         0.9767  
54   0.075574         0.9765  
   Optimizer  Learning Rate Momentum       Epsilon Nesterov Batch Size  \
0       Adam        0.00001        0  1.000000e-08    False         64   
1       Adam        0.00001        0  1.000000e-08    False        256   
2       Adam        0.00001        0  1.000000e-08     True         64   
3       Adam        0.00001        

# Parameters results:
- Initialization method: GlorotUniform 
- Activation function: Relu
- Optimizer: RMSdrop
- Regulazation technique: Dropout
- Optimizer + hyperparameters (Learning rate, momentum, epsilon, nesterov, batch size): SGD, 0.1, 0.9, 1.000000e-04, True, 256